# Multi-Agent Orchestration with OpenAI Agents SDK: Financial Portfolio Analysis Example

## Introduction
This project is a demo of multi-agent orchestration with OpenAI Agents SDK. It shows how to use the OpenAI Agents SDK to build a multi-agent system that can work together to complete a task mimicing the collaboration of the patient advocacy department.

## Setup

In [1]:
# Install dependencies
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Before running the workflow, set the following environment variables:
OPENAI_API_KEY

In [1]:
import os

missing = []
if not os.environ.get('OPENAI_API_KEY'):
    missing.append('OPENAI_API_KEY')
if missing:
    print(f"Missing environment variable(s): {', '.join(missing)}. Please set them before running the workflow.")
else:
    print("All required API keys are set.")

All required API keys are set.


## Running the orchestration workflow

The workflow is kicked off by sending a user request to the Head Portfolio Manager (PM) agent. The PM agent orchestrates the entire process, delegating to specialist agents and tools as needed. You can monitor the workflow in real time using OpenAI Traces, which provide detailed visibility into every agent and tool call.

In [47]:
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

In [4]:
import json
import os
from pathlib import Path
from contextlib import AsyncExitStack
from agents import Runner, add_trace_processor, trace
from agents.tracing.processors import BatchTraceProcessor
from utils import FileSpanExporter, output_file
from special_agents.config import build_pa_agents
import asyncio


add_trace_processor(BatchTraceProcessor(FileSpanExporter()))

async def run_workflow():
    if "OPENAI_API_KEY" not in os.environ:
        raise EnvironmentError("OPENAI_API_KEY not set — set it as an environment variable before running.")

    question = (
        "If we want to add a new support line for a new program, what do we need to do?"
    )
    bundle = build_pa_agents()

    async with AsyncExitStack() as stack:
        for agent in [getattr(bundle, "pa_director", None), getattr(bundle, "bpo_tools_manager", None), getattr(bundle, "bpo_trainer", None), getattr(bundle, "bpo_manager", None), getattr(bundle, "voc_manager", None)]:
            if agent is None:
                continue
            for server in getattr(agent, "mcp_servers", []):
                await server.connect()
                await stack.enter_async_context(server)

        print("Running multi-agent workflow with tracing enabled...\n")
        with trace(
            "PA Workflow",
            metadata={"question": question[:512]}
        ) as workflow_trace:
            print(
                f"\n🔗 View the trace in the OpenAI console: "
                f"https://platform.openai.com/traces/trace?trace_id={workflow_trace.trace_id}\n"
            )

            response = None
            try:
                response = await asyncio.wait_for(
                    Runner.run(bundle.pa_director, question, max_turns=40),
                    timeout=1200
                )
            except asyncio.TimeoutError:
                print("\n❌ Workflow timed out after 20 minutes.")

            report_path = None
            try:
                if hasattr(response, 'final_output'):
                    output = response.final_output
                    if isinstance(output, str):
                        print(output)
                        data = json.loads(output)
                        if isinstance(data, dict) and 'file' in data:
                            report_path = output_file(data['file'])
            except Exception as e:
                print(f"Could not parse report path: {e}")

            print(f"Workflow Completed Response from Agent: {response.final_output if hasattr(response, 'final_output') else response}")

# In a Jupyter notebook cell, run:
await run_workflow()

Running multi-agent workflow with tracing enabled...


🔗 View the trace in the OpenAI console: https://platform.openai.com/traces/trace?trace_id=trace_e0dba653efff464d9adc8b4452d5141b

To add a new support line for a new program, you’ll need to coordinate across several teams and systems. Here’s a comprehensive overview of what needs to be done, based on input from each specialist area:

---

### 1. BPO Tools & Systems Setup
- **Talkdesk (Telephony):** Request and provision a new phone number, configure call routing/IVR, assign agents, and test the line.
- **Zendesk (Ticketing):** Set up a new support channel, configure triggers and automations, update views/macros, and ensure integration with Talkdesk for call logging.
- **Knowledge Base:** Update or create SOPs and resources for the new program.
- **Other Tools:** Update chatbot flows (if applicable), link post-call surveys, and confirm agent access to all necessary internal tools.

---

### 2. Training & Knowledge Management
- **Tra